# 自己的情感分析任务
针对自己的数据集的情感分析任务，加载之前训练好的预训练模型，用自己的游记文本数据再次训练模型，学习游记文本中的语义。<br/>
加入其他网络模型，比如加入LSTM

## 参数设置和变量设置

In [1]:
check_point = "schen/longformer-chinese-base-4096"
# check_point = "/home/chenli/pre_model/best_model/Longformer_SentimentAnalysis"
# check_point = "bert-base-chinese"
batch_size = 2 # 每一批次的数量

## 加载数据

In [2]:
from datasets import load_dataset
from datasets import load_from_disk
# 加载一个评估标准，默认的评估标准
from datasets import load_metric

In [3]:
train_dataset = load_dataset('csv',data_files='../data/MyDataset/data2/train_dataset.csv',split='train')
valid_dataset = load_dataset('csv',data_files='../data/MyDataset/data2/valid_dataset.csv',split='train')
test_dataset = load_dataset('csv',data_files='../data/MyDataset/data2/test_dataset.csv',split='train')

Using custom data configuration default-5602383f9cde0ea3
Reusing dataset csv (/home/chenli/.cache/huggingface/datasets/csv/default-5602383f9cde0ea3/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
Using custom data configuration default-062c84d526dcea84
Reusing dataset csv (/home/chenli/.cache/huggingface/datasets/csv/default-062c84d526dcea84/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
Using custom data configuration default-0f8395db45727ded
Reusing dataset csv (/home/chenli/.cache/huggingface/datasets/csv/default-0f8395db45727ded/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


In [4]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 2755
})

## 数据预处理

In [5]:
from transformers import AutoTokenizer
    
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(check_point)

In [6]:
tokenizer

PreTrainedTokenizerFast(name_or_path='schen/longformer-chinese-base-4096', vocab_size=21128, model_max_len=4096, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [6]:
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-chinese', vocab_size=21128, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [7]:
# 分词，Longformer模型
def preprocess_function(data):
    return tokenizer(data['text'],padding='max_length',max_length=1500,truncation=True)

In [7]:
# 分词，BERT模型
def preprocess_function(data):
    return tokenizer(data['text'],padding='max_length',max_length=512,truncation=True)

In [8]:
encoded_train_dataset = train_dataset.map(function=preprocess_function,
                     batched=True,
                     remove_columns=['text'])
encoded_train_dataset = encoded_train_dataset.rename_column("label", "labels")
encoded_train_dataset

Loading cached processed dataset at /home/chenli/.cache/huggingface/datasets/csv/default-5602383f9cde0ea3/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-5fe571b7d0f7641f.arrow


Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2755
})

In [9]:
encoded_valid_dataset = valid_dataset.map(function=preprocess_function,
                     batched=True,
                     remove_columns=['text'])
encoded_valid_dataset = encoded_valid_dataset.rename_column("label", "labels")
encoded_valid_dataset

Loading cached processed dataset at /home/chenli/.cache/huggingface/datasets/csv/default-062c84d526dcea84/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-6f90ea17c5ea978e.arrow


Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 344
})

In [10]:
encoded_test_dataset = test_dataset.map(function=preprocess_function,
                     batched=True,
                     remove_columns=['text'])
encoded_test_dataset = encoded_test_dataset.rename_column("label", "labels")
encoded_test_dataset

Loading cached processed dataset at /home/chenli/.cache/huggingface/datasets/csv/default-0f8395db45727ded/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-f44dd11c97e01e8e.arrow


Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 345
})

## 数据集加载器

In [11]:
import torch
from transformers.data.data_collator import DataCollatorWithPadding
train_dataloader = torch.utils.data.DataLoader(dataset = encoded_train_dataset,batch_size=batch_size,collate_fn=DataCollatorWithPadding(tokenizer),shuffle=True,drop_last=True)
valid_dataloader = torch.utils.data.DataLoader(dataset = encoded_valid_dataset,batch_size=batch_size,collate_fn=DataCollatorWithPadding(tokenizer),shuffle=True,drop_last=True)
test_dataloader = torch.utils.data.DataLoader(dataset = encoded_test_dataset,batch_size=batch_size,collate_fn=DataCollatorWithPadding(tokenizer),shuffle=True,drop_last=True)

In [12]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 设置使用第几块GPU
id = 0
torch.cuda.set_device(id)
print(f'Using {device} device')
for batch,data in enumerate(test_dataloader,start=1):
    labels = data.labels.to(device)
    input_ids = data.input_ids.to(device)
    token_type_ids = data.token_type_ids.to(device)
    attention_mask = data.attention_mask.to(device)
    print(labels,input_ids,token_type_ids,attention_mask)
    break

Using cuda device
tensor([1, 1], device='cuda:1') tensor([[ 101, 3946, 2428,  ...,    0,    0,    0],
        [ 101, 7350, 1488,  ..., 1282, 1368,  102]], device='cuda:1') tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:1') tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]], device='cuda:1')


## 全连接层
使用任务一的模型，对自己的数据集进行训练，只使用简单地线性分类

In [13]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch import nn
from transformers import AutoModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 设置使用几块GPU
id = 0
torch.cuda.set_device(id)
print(f'Using {device} device')
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # 这个可以拿到预训练模型最后一层的结果
        self.longformer = AutoModel.from_pretrained(check_point)
        self.linear = nn.Linear(768, 2)

    def forward(self,input_ids,token_type_ids,attention_mask):
        # 取最后一层的第一个，因为我们希望拿到的是整句话的一个语义
        output = self.longformer(input_ids,token_type_ids,attention_mask).last_hidden_state[:,0] # 维度 [batch,seq,hidden_size]
        # 然后输送给分类的linear层
        output = self.linear(output)
        return output

# 如果显卡的话就使用显卡
model = NeuralNetwork().to(device)
print(model)

Using cuda device


NeuralNetwork(
  (longformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [13]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch import nn
from transformers import AutoModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # 这个可以拿到预训练模型最后一层的结果
        self.longformer = AutoModel.from_pretrained(check_point)
        self.linear = nn.Sequential(
            nn.Linear(768,512),
            torch.nn.Dropout(0.5),
            torch.nn.ReLU(),
            nn.Linear(512,2)
        )

    def forward(self,input_ids,token_type_ids,attention_mask):
        # 取最后一层的第一个，因为我们希望拿到的是整句话的一个语义
        output = self.longformer(input_ids,token_type_ids,attention_mask).last_hidden_state[:,0] # 维度 [batch,seq,hidden_size]
        # 然后输送给分类的linear层
        output = self.linear(output)
        return output

# 如果显卡的话就使用显卡
model = NeuralNetwork().to(device)
print(model)

Using cuda device


Some weights of the model checkpoint at schen/longformer-chinese-base-4096 were not used when initializing BertModel: ['bert.encoder.layer.0.attention.self.query_global.weight', 'bert.encoder.layer.0.attention.self.query_global.bias', 'bert.encoder.layer.0.attention.self.key_global.weight', 'bert.encoder.layer.0.attention.self.key_global.bias', 'bert.encoder.layer.0.attention.self.value_global.weight', 'bert.encoder.layer.0.attention.self.value_global.bias', 'bert.encoder.layer.1.attention.self.query_global.weight', 'bert.encoder.layer.1.attention.self.query_global.bias', 'bert.encoder.layer.1.attention.self.key_global.weight', 'bert.encoder.layer.1.attention.self.key_global.bias', 'bert.encoder.layer.1.attention.self.value_global.weight', 'bert.encoder.layer.1.attention.self.value_global.bias', 'bert.encoder.layer.2.attention.self.query_global.weight', 'bert.encoder.layer.2.attention.self.query_global.bias', 'bert.encoder.layer.2.attention.self.key_global.weight', 'bert.encoder.layer.

NeuralNetwork(
  (longformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

## RNN

单向RNN

In [11]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch import nn
from transformers import AutoModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} device')
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # 这个可以拿到预训练模型最后一层的结果
        self.longformer = AutoModel.from_pretrained(check_point)
        # 接一个BiGRU
        self.rnn = nn.RNN(input_size=768,hidden_size=512,batch_first=True,dropout=0.5)
        # 这里可以接分类层，输入768维，最后分为2个类别
        # 这里可以添加其他网络模型，提升效果
        # 加多一个线性层
        self.linear = nn.Sequential(
            nn.Linear(512,512),
            torch.nn.Dropout(0.5),
            torch.nn.ReLU(),
            nn.Linear(512,2)
        )
        

    def forward(self,input_ids,token_type_ids,attention_mask):
        # 取最后一层的第一个，因为我们希望拿到的是整句话的一个语义
        output = self.longformer(input_ids,token_type_ids,attention_mask).last_hidden_state # 维度 [batch,seq,hidden_size]
        output,h_n = self.rnn(output)
        output = output[:,-1,:] # [batch,1024]
        # 然后输送给分类的linear层
        output = self.linear(output)
        return output

# 如果显卡的话就使用显卡
model = NeuralNetwork().to(device)
print(model)

Using cuda device


Some weights of the model checkpoint at schen/longformer-chinese-base-4096 were not used when initializing BertModel: ['bert.encoder.layer.10.attention.self.query_global.bias', 'bert.encoder.layer.1.attention.self.query_global.bias', 'bert.encoder.layer.1.attention.self.value_global.bias', 'bert.encoder.layer.5.attention.self.query_global.bias', 'bert.encoder.layer.5.attention.self.value_global.weight', 'cls.predictions.transform.LayerNorm.weight', 'bert.encoder.layer.8.attention.self.query_global.weight', 'bert.encoder.layer.2.attention.self.value_global.bias', 'bert.encoder.layer.8.attention.self.query_global.bias', 'bert.encoder.layer.7.attention.self.query_global.weight', 'bert.encoder.layer.7.attention.self.value_global.bias', 'bert.encoder.layer.8.attention.self.value_global.weight', 'bert.encoder.layer.3.attention.self.value_global.weight', 'bert.encoder.layer.11.attention.self.value_global.weight', 'bert.encoder.layer.7.attention.self.key_global.bias', 'cls.seq_relationship.wei

NeuralNetwork(
  (longformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

双向RNN

In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch import nn
from transformers import AutoModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} device')
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # 这个可以拿到预训练模型最后一层的结果
        self.longformer = AutoModel.from_pretrained(check_point)
        # 接一个BiGRU
        self.rnn = nn.RNN(input_size=768,hidden_size=512,batch_first=True,bidirectional=True,dropout=0.5)
        # 这里可以接分类层，输入768维，最后分为2个类别
        # 这里可以添加其他网络模型，提升效果
        # 加多一个线性层
        self.linear = nn.Sequential(
            nn.Linear(1024,512),
            torch.nn.Dropout(0.5),
            torch.nn.ReLU(),
            nn.Linear(512,2)
        )
        

    def forward(self,input_ids,token_type_ids,attention_mask):
        # 取最后一层的第一个，因为我们希望拿到的是整句话的一个语义
        output = self.longformer(input_ids,token_type_ids,attention_mask).last_hidden_state # 维度 [batch,seq,hidden_size]
        output,h_n = self.rnn(output)
        output = output[:,-1,:] # [batch,1024]
        # 然后输送给分类的linear层
        output = self.linear(output)
        return output

# 如果显卡的话就使用显卡
model = NeuralNetwork().to(device)
print(model)

## LSTM
针对自己数据集进行微调 <br/>
加入LSTM网络模型

In [13]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch import nn
from transformers import AutoModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # 这个可以拿到预训练模型最后一层的结果
        self.longformer = AutoModel.from_pretrained(check_point)
        # 接一个BiGRU
        self.lstm = nn.LSTM(input_size=768,hidden_size=512,batch_first=True,bidirectional=True)
        # 这里可以接分类层，输入768维，最后分为2个类别
        # 这里可以添加其他网络模型，提升效果
        # 加多一个线性层
        self.linear = nn.Linear(1024,2)

    def forward(self,input_ids,token_type_ids,attention_mask):
        # 取最后一层的第一个，因为我们希望拿到的是整句话的一个语义
        output = self.longformer(input_ids,token_type_ids,attention_mask).last_hidden_state # 维度 [batch,seq,hidden_size]
        output,h_n = self.lstm(output)
        output = output[:,-1,:] # [batch,1024]
        # 然后输送给分类的linear层
        output = self.linear(output)
        return output

# 如果显卡的话就使用显卡
model = NeuralNetwork().to(device)
model.eval()

Using cuda device


NeuralNetwork(
  (longformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [12]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch import nn
from transformers import AutoModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # 这个可以拿到预训练模型最后一层的结果
        self.longformer = AutoModel.from_pretrained(check_point)
        # 接一个BiGRU
        self.lstm = nn.LSTM(input_size=768,hidden_size=512,batch_first=True,bidirectional=True,dropout=0.5)
        # 这里可以接分类层，输入768维，最后分为2个类别
        # 这里可以添加其他网络模型，提升效果
        # 加多一个线性层
        self.linear = self.linear = nn.Sequential(
            nn.Linear(1024,512),
            torch.nn.Dropout(0.5),
            torch.nn.ReLU(),
            nn.Linear(512,2)
        )

    def forward(self,input_ids,token_type_ids,attention_mask):
        # 取最后一层的第一个，因为我们希望拿到的是整句话的一个语义
        output = self.longformer(input_ids,token_type_ids,attention_mask).last_hidden_state # 维度 [batch,seq,hidden_size]
        output,h_n = self.lstm(output)
        output = output[:,-1,:] # [batch,1024]
        # 然后输送给分类的linear层
        output = self.linear(output)
        return output

# 如果显卡的话就使用显卡
model = NeuralNetwork().to(device)
print(model)

Using cuda device


Some weights of the model checkpoint at schen/longformer-chinese-base-4096 were not used when initializing BertModel: ['bert.encoder.layer.5.attention.self.key_global.weight', 'cls.predictions.transform.dense.bias', 'bert.encoder.layer.11.attention.self.value_global.weight', 'bert.encoder.layer.8.attention.self.key_global.weight', 'bert.encoder.layer.6.attention.self.query_global.bias', 'bert.encoder.layer.6.attention.self.query_global.weight', 'bert.encoder.layer.4.attention.self.value_global.bias', 'bert.encoder.layer.5.attention.self.value_global.weight', 'bert.encoder.layer.4.attention.self.key_global.bias', 'cls.predictions.decoder.bias', 'bert.encoder.layer.10.attention.self.query_global.weight', 'cls.seq_relationship.bias', 'bert.encoder.layer.10.attention.self.key_global.bias', 'cls.predictions.transform.dense.weight', 'bert.encoder.layer.7.attention.self.value_global.bias', 'bert.encoder.layer.9.attention.self.value_global.weight', 'bert.encoder.layer.1.attention.self.key_glob

NeuralNetwork(
  (longformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

单层LSTM

In [12]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch import nn
from transformers import AutoModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'
id = 1
torch.cuda.set_device(id)
print(f'Using {device} device')
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # 这个可以拿到预训练模型最后一层的结果
        self.longformer = AutoModel.from_pretrained(check_point)
        # 接一个BiGRU
        self.lstm = nn.LSTM(input_size=768,hidden_size=512,batch_first=True,dropout=0.5)
        # 这里可以接分类层，输入768维，最后分为2个类别
        # 这里可以添加其他网络模型，提升效果
        # 加多一个线性层
        self.linear = self.linear = nn.Sequential(
            nn.Linear(512,512),
            torch.nn.Dropout(0.5),
            torch.nn.ReLU(),
            nn.Linear(512,2)
        )

    def forward(self,input_ids,token_type_ids,attention_mask):
        # 取最后一层的第一个，因为我们希望拿到的是整句话的一个语义
        output = self.longformer(input_ids,token_type_ids,attention_mask).last_hidden_state # 维度 [batch,seq,hidden_size]
        output,h_n = self.lstm(output)
        output = output[:,-1,:] # [batch,1024]
        # 然后输送给分类的linear层
        output = self.linear(output)
        return output

# 如果显卡的话就使用显卡
model = NeuralNetwork().to(device)
print(model)

Using cuda device


Some weights of the model checkpoint at schen/longformer-chinese-base-4096 were not used when initializing BertModel: ['bert.encoder.layer.7.attention.self.key_global.weight', 'bert.encoder.layer.6.attention.self.value_global.bias', 'bert.encoder.layer.11.attention.self.query_global.bias', 'bert.encoder.layer.2.attention.self.value_global.bias', 'bert.encoder.layer.0.attention.self.query_global.weight', 'bert.encoder.layer.8.attention.self.value_global.bias', 'bert.encoder.layer.5.attention.self.query_global.bias', 'bert.encoder.layer.11.attention.self.key_global.bias', 'bert.encoder.layer.7.attention.self.value_global.bias', 'bert.encoder.layer.6.attention.self.query_global.weight', 'bert.encoder.layer.9.attention.self.query_global.weight', 'bert.encoder.layer.5.attention.self.query_global.weight', 'bert.encoder.layer.3.attention.self.key_global.bias', 'bert.encoder.layer.9.attention.self.key_global.weight', 'bert.encoder.layer.6.attention.self.query_global.bias', 'bert.encoder.layer.

NeuralNetwork(
  (longformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

## GRU网络

In [12]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch import nn
from transformers import AutoModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'
id = 0
torch.cuda.set_device(id)
print(f'Using {device} device')
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # 这个可以拿到预训练模型最后一层的结果
        self.longformer = AutoModel.from_pretrained(check_point)
        # 接一个BiGRU
        self.gru = nn.GRU(input_size=768,hidden_size=512,batch_first=True,bidirectional=True,dropout=0.5)
        # 这里可以接分类层，输入768维，最后分为2个类别
        # 这里可以添加其他网络模型，提升效果
        # 加多一个线性层
        self.linear = nn.Sequential(
            nn.Linear(1024,512),
            torch.nn.Dropout(0.5),
            torch.nn.ReLU(),
            nn.Linear(512,2)
        )
        

    def forward(self,input_ids,token_type_ids,attention_mask):
        # 取最后一层的第一个，因为我们希望拿到的是整句话的一个语义
        output = self.longformer(input_ids,token_type_ids,attention_mask).last_hidden_state # 维度 [batch,seq,hidden_size]
        print(output)
        output,h_n = self.gru(output)
        print(output)
        output = output[:,-1,:] # [batch,1024]
        print(output)
        # 然后输送给分类的linear层
        output = self.linear(output)
        print(output)
        return output

# 如果显卡的话就使用显卡
model = NeuralNetwork().to(device)
model.eval()

Using cuda device


Some weights of the model checkpoint at schen/longformer-chinese-base-4096 were not used when initializing BertModel: ['bert.encoder.layer.8.attention.self.query_global.weight', 'bert.encoder.layer.1.attention.self.query_global.bias', 'bert.encoder.layer.6.attention.self.query_global.weight', 'bert.encoder.layer.2.attention.self.value_global.weight', 'cls.predictions.transform.LayerNorm.weight', 'bert.encoder.layer.3.attention.self.key_global.weight', 'bert.encoder.layer.11.attention.self.value_global.weight', 'bert.encoder.layer.7.attention.self.key_global.bias', 'bert.encoder.layer.0.attention.self.value_global.bias', 'bert.encoder.layer.7.attention.self.query_global.weight', 'cls.predictions.decoder.bias', 'bert.encoder.layer.1.attention.self.key_global.weight', 'bert.encoder.layer.4.attention.self.key_global.weight', 'bert.encoder.layer.5.attention.self.query_global.bias', 'bert.encoder.layer.4.attention.self.query_global.weight', 'bert.encoder.layer.9.attention.self.key_global.bia

NeuralNetwork(
  (longformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

单向的GRU

In [13]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch import nn
from transformers import AutoModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'
id = 1
torch.cuda.set_device(id)
print(f'Using {device} device')
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # 这个可以拿到预训练模型最后一层的结果
        self.longformer = AutoModel.from_pretrained(check_point)
        # 接一个BiGRU
        self.gru = nn.GRU(input_size=768,hidden_size=512,batch_first=True,dropout=0.5)
        # 这里可以接分类层，输入768维，最后分为2个类别
        # 这里可以添加其他网络模型，提升效果
        # 加多一个线性层
        self.linear = nn.Sequential(
            nn.Linear(512,512),
            torch.nn.Dropout(0.5),
            torch.nn.ReLU(),
            nn.Linear(512,2)
        )
        

    def forward(self,input_ids,token_type_ids,attention_mask):
        # 取最后一层的第一个，因为我们希望拿到的是整句话的一个语义
        output = self.longformer(input_ids,token_type_ids,attention_mask).last_hidden_state # 维度 [batch,seq,hidden_size]
        output,h_n = self.gru(output)
        output = output[:,-1,:] # [batch,1024]
        # 然后输送给分类的linear层
        output = self.linear(output)
        return output

# 如果显卡的话就使用显卡
model = NeuralNetwork().to(device)
model.eval()

Using cuda device


Some weights of the model checkpoint at schen/longformer-chinese-base-4096 were not used when initializing BertModel: ['bert.encoder.layer.8.attention.self.value_global.bias', 'bert.encoder.layer.7.attention.self.value_global.bias', 'bert.encoder.layer.5.attention.self.query_global.weight', 'bert.encoder.layer.6.attention.self.key_global.weight', 'bert.encoder.layer.2.attention.self.value_global.weight', 'bert.encoder.layer.1.attention.self.value_global.weight', 'bert.encoder.layer.9.attention.self.key_global.bias', 'bert.encoder.layer.9.attention.self.value_global.bias', 'cls.predictions.bias', 'bert.encoder.layer.0.attention.self.query_global.bias', 'cls.predictions.decoder.bias', 'bert.encoder.layer.11.attention.self.key_global.bias', 'bert.encoder.layer.1.attention.self.key_global.weight', 'bert.encoder.layer.0.attention.self.value_global.bias', 'bert.encoder.layer.0.attention.self.key_global.bias', 'bert.encoder.layer.6.attention.self.query_global.bias', 'bert.encoder.layer.6.atte

NeuralNetwork(
  (longformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

## 模型训练

In [15]:
from tqdm.auto import tqdm # 显示它的进度条，会更好看点

# 参数解释
# dataloader ： 批量数据的loader
# model : 定义的模型
# loss_fn ： 定义的损失函数
# optimizer ：优化器
# lr_scheduler ： 学习率根据步数会下降，动态变化的。如果用一个固定的学习率，其实是没有这种随着迭代次数下降的效果好的
# epoch ：训练的轮次
# total_loss ：整体loss的情况
def train_loop(dataloader, model, loss_fn, optimizer, lr_scheduler, epoch, total_loss):
    progress_bar = tqdm(range(len(dataloader)))
    progress_bar.set_description(f'loss: {0:>7f}')
    finish_batch_num = (epoch-1)*len(dataloader)
    
    model.train()
    for batch, data in enumerate(dataloader,start=1):
        labels = data.labels.to(device)
        input_ids = data.input_ids.to(device)
        token_type_ids = data.token_type_ids.to(device)
        attention_mask = data.attention_mask.to(device)
        pred = model(input_ids,token_type_ids,attention_mask)
        print(pred)
        loss = loss_fn(pred, labels)

        
        loss.backward() # 向后传播
        optimizer.step() # 算完梯度下降之后更改参数
        lr_scheduler.step() # 对学习率进行调整
        optimizer.zero_grad() # 把之前的梯度都清掉

        total_loss += loss.item() # 统计一下整体的loss
        # batch=2时，每600输出一次loss
        # batch=4时，可以每300输出一次loss
        progress_bar.set_description(f'loss: {total_loss/(finish_batch_num + batch):>7f}')
        progress_bar.update(1)
        
        
        # running_loss += loss.item() 
        # 每600输出一次loss
        # if batch % 600 == 599:
            # print('[%d,%5d] running_loss:%.3f' % (epoch,batch+1,running_loss/600))
            # 每600一组数据跑完，清零
            # running_loss = 0.0
    return total_loss

def test_loop(dataloader, model, mode='Test'):
    assert mode in ['Valid', 'Test']
    size = len(dataloader.dataset)
    correct = 0

    model.eval()
    with torch.no_grad():
        for batch,data in enumerate(dataloader,start=1):
            labels = data.labels.to(device)
            input_ids = data.input_ids.to(device)
            token_type_ids = data.token_type_ids.to(device)
            attention_mask = data.attention_mask.to(device)
            pred = model(input_ids,token_type_ids,attention_mask)
            print(pred)
            correct += (pred.argmax(1) == labels).type(torch.float).sum().item()

    correct /= size
    print(f"{mode} Accuracy: {(100*correct):>0.1f}%\n")
    return correct

In [16]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 1 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

Epoch 1/1
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 206.00 MiB (GPU 0; 14.76 GiB total capacity; 13.34 GiB already allocated; 135.75 MiB free; 13.47 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# 开始训练

# 全连接层的训练 

## 20221117 训练
使用任务一，对自己的游记文本数据集训练，不加入任务网络模型，单纯使用线性层分类

In [15]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 10 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

Epoch 1/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 96.5%

saving new weights...

Epoch 2/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 96.8%

saving new weights...

Epoch 3/10
-------------------------------


...

Valid Accuracy: 97.1%

Epoch 5/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 96.5%

Epoch 6/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

KeyboardInterrupt: 

## 20221118 训练
self.longformer = AutoModel.from_pretrained(check_point) <br/>
self.linear = nn.Sequential( <br/>
    nn.Linear(768,512),<br/>
    torch.nn.Dropout(0.5),<br/>
    torch.nn.ReLU(),<br/>
    nn.Linear(512,2)<br/>
)

In [15]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 10 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

Epoch 1/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.6%

saving new weights...

Epoch 2/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.9%

saving new weights...

Epoch 3/10
-------------------------------


...

Valid Accuracy: 96.2%

saving new weights...

Epoch 5/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 97.1%

saving new weights...

Epoch 6/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

KeyboardInterrupt: 

## RNN训练

### 20221216
单向RNN

In [13]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 10 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

Epoch 1/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 94.8%

saving new weights...

Epoch 2/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.1%

saving new weights...

Epoch 3/10
-------------------------------


...

Valid Accuracy: 95.6%

Epoch 9/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 96.2%

saving new weights...

Epoch 10/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.9%

Done!


In [14]:
valid_acc = test_loop(test_dataloader, model, mode='Test')
print(valid_acc)
print("Done!")

Test Accuracy: 95.4%

0.9536231884057971
Done!


双向RNN

In [ ]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 10 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

# LSTM训练

## 20221116 训练
没有使用任务一，加入LSTM网络模型

In [14]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 10 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

Epoch 1/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

ValueError: too many values to unpack (expected 2)

## 20221117 训练
没有使用任务一，LSTM网络的加入

In [14]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 10 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

Epoch 1/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.6%

saving new weights...

Epoch 2/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.3%

Epoch 3/10
-------------------------------


...

Valid Accuracy: 94.8%

Epoch 9/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.1%

Epoch 10/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.6%

Done!


## 20221117 训练
使用任务一，加入LSTM网络进行训练

In [15]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 10 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

Epoch 1/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.9%

saving new weights...

Epoch 2/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 96.5%

saving new weights...

Epoch 3/10
-------------------------------


...

Valid Accuracy: 95.9%

Epoch 9/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 96.8%

saving new weights...

Epoch 10/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 96.8%

Done!


## 20221117训练
任务一的基础上，加入LSTM，加多一层linear层，看看效果

In [15]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 5 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

Epoch 1/5
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.6%

saving new weights...

Epoch 2/5
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.6%

Epoch 3/5
-------------------------------


...

Valid Accuracy: 96.2%

saving new weights...

Epoch 4/5
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.3%

Epoch 5/5
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.3%

Done!


## 20221119 训练
self.longformer = AutoModel.from_pretrained(check_point)
        
self.lstm = nn.LSTM(input_size=768,hidden_size=512,batch_first=True,bidirectional=True,dropout=0.5)

self.linear = self.linear = nn.Sequential(
    nn.Linear(1024,512),
    torch.nn.Dropout(0.5),
    torch.nn.ReLU(),
    nn.Linear(512,2)
)

In [14]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 5 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

Epoch 1/5
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.1%

saving new weights...

Epoch 2/5
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.6%

saving new weights...

Epoch 3/5
-------------------------------


...

Valid Accuracy: 95.6%

Epoch 4/5
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.6%

Epoch 5/5
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



直接验证

In [15]:
valid_acc = test_loop(test_dataloader, model, mode='Test')
print(valid_acc)
print("Done!")

Test Accuracy: 95.4%

0.9536231884057971
Done!


## 20221215
Longformer + BiLSTM

In [14]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 10 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

Epoch 1/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.1%

saving new weights...

Epoch 2/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.6%

saving new weights...

Epoch 3/10
-------------------------------


...

Valid Accuracy: 96.2%

Epoch 9/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 96.8%

saving new weights...

Epoch 10/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 96.5%

Done!


In [15]:
valid_acc = test_loop(test_dataloader, model, mode='Test')
print(valid_acc)
print("Done!")

Test Accuracy: 95.4%

0.9536231884057971
Done!


## 20221204 训练
单向LSTM

In [14]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 10 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

Epoch 1/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 87.2%

saving new weights...

Epoch 2/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.3%

saving new weights...

Epoch 3/10
-------------------------------


...

Valid Accuracy: 47.4%

Epoch 7/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 46.8%

Epoch 8/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [15]:
valid_acc = test_loop(test_dataloader, model, mode='Test')
print(valid_acc)
print("Done!")

Test Accuracy: 43.5%

0.43478260869565216
Done!


对测试集进行测试

In [13]:
# 加载模型参数
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load('/home/chenli/pre_model/experiment_model/Longformer+LSTM/epoch_2_valid_acc_95.3_model_weights.bin'))
model.eval()

Some weights of the model checkpoint at schen/longformer-chinese-base-4096 were not used when initializing BertModel: ['bert.encoder.layer.7.attention.self.key_global.weight', 'bert.encoder.layer.6.attention.self.value_global.bias', 'bert.encoder.layer.11.attention.self.query_global.bias', 'bert.encoder.layer.2.attention.self.value_global.bias', 'bert.encoder.layer.0.attention.self.query_global.weight', 'bert.encoder.layer.8.attention.self.value_global.bias', 'bert.encoder.layer.5.attention.self.query_global.bias', 'bert.encoder.layer.11.attention.self.key_global.bias', 'bert.encoder.layer.7.attention.self.value_global.bias', 'bert.encoder.layer.6.attention.self.query_global.weight', 'bert.encoder.layer.9.attention.self.query_global.weight', 'bert.encoder.layer.5.attention.self.query_global.weight', 'bert.encoder.layer.3.attention.self.key_global.bias', 'bert.encoder.layer.9.attention.self.key_global.weight', 'bert.encoder.layer.6.attention.self.query_global.bias', 'bert.encoder.layer.

NeuralNetwork(
  (longformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [14]:
def test_loop(dataloader, model, mode='Test'):
    assert mode in ['Valid', 'Test']
    size = len(dataloader.dataset)
    correct = 0

    model.eval()
    with torch.no_grad():
        for batch,data in enumerate(dataloader,start=1):
            labels = data.labels.to(device)
            input_ids = data.input_ids.to(device)
            token_type_ids = data.token_type_ids.to(device)
            attention_mask = data.attention_mask.to(device)
            pred = model(input_ids,token_type_ids,attention_mask)
            correct += (pred.argmax(1) == labels).type(torch.float).sum().item()

    correct /= size
    print(f"{mode} Accuracy: {(100*correct):>0.1f}%\n")
    return correct

In [15]:
valid_acc = test_loop(test_dataloader, model, mode='Test')
print(valid_acc)
print("Done!")

Test Accuracy: 95.7%

0.9565217391304348
Done!


## GRU训练

## 20221118训练
self.longformer = AutoModel.from_pretrained(check_point)<br/>
        
self.gru = nn.GRU(input_size=768,hidden_size=512,batch_first=True,bidirectional=True,dropout=0.5)<br/>

self.linear = nn.Sequential(
    nn.Linear(1024,512),
    torch.nn.Dropout(0.5),
    torch.nn.ReLU(),
    nn.Linear(512,2)
)

In [18]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 5 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

Epoch 1/5
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.1%

saving new weights...

Epoch 2/5
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.6%

saving new weights...

Epoch 3/5
-------------------------------


...

Valid Accuracy: 96.8%

saving new weights...

Epoch 4/5
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 97.1%

saving new weights...

Epoch 5/5
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 96.5%

Done!


直接验证

In [19]:
valid_acc = test_loop(test_dataloader, model, mode='Test')
print(valid_acc)
print("Done!")

Test Accuracy: 96.2%

0.9623188405797102
Done!


## 20221119 训练
任务一的基础上，加入GRU网络模型

In [12]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 5 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

Epoch 1/5
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.3%

saving new weights...

Epoch 2/5
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.6%

saving new weights...

Epoch 3/5
-------------------------------


...

Valid Accuracy: 95.3%

Epoch 4/5
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 96.8%

saving new weights...

Epoch 5/5
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 96.5%

Done!


## 20221128 训练
BERT + BiGRU

In [16]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 10 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

Epoch 1/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 96.5%

saving new weights...

Epoch 2/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.6%

Epoch 3/10
-------------------------------


...

Valid Accuracy: 97.4%

Epoch 5/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 97.1%

Epoch 6/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

KeyboardInterrupt: 

## 20221128 训练
Longformer + GRU <br>
单向

In [15]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 10 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

Epoch 1/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.6%

saving new weights...

Epoch 2/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.9%

saving new weights...

Epoch 3/10
-------------------------------


...

Valid Accuracy: 95.9%

Epoch 9/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 96.2%

Epoch 10/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 96.2%

Done!


In [16]:
valid_acc = test_loop(test_dataloader, model, mode='Test')
print(valid_acc)
print("Done!")

Test Accuracy: 95.7%

0.9565217391304348
Done!


## 20221215
Longformer + BiGRU

In [14]:
from transformers import AdamW, get_scheduler

learning_rate = 1e-5 # 定义学习率
epoch_num = 10 # 轮次定义

loss_fn = nn.CrossEntropyLoss() # 损失函数，交叉熵
optimizer = AdamW(model.parameters(), lr=learning_rate) # Adamw一个常用的优化器
lr_scheduler = get_scheduler(
    "linear",# 使用线性的方式，慢慢往下降
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_acc = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, t+1, total_loss)
    valid_acc = test_loop(valid_dataloader, model, mode='Valid')
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('saving new weights...\n')
        # 保存模型
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_acc_{(100*valid_acc):0.1f}_model_weights.bin')
print("Done!")

Epoch 1/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.9%

saving new weights...

Epoch 2/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.6%

Epoch 3/10
-------------------------------


...

Valid Accuracy: 95.6%

Epoch 9/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 96.2%

saving new weights...

Epoch 10/10
-------------------------------


  0%|          | 0/1377 [00:00<?, ?it/s]

Valid Accuracy: 95.9%

Done!


In [15]:
valid_acc = test_loop(test_dataloader, model, mode='Test')
print(valid_acc)
print("Done!")

Test Accuracy: 95.1%

0.9507246376811594
Done!


In [1]:
import torch
from transformers import AutoModel
from thop import profile


model = AutoModel.from_pretrained("bert-base-chinese") 
input_shape = (1, 512) # some example input shape
input_tensor = torch.rand(*input_shape)

flops, params = profile(model, inputs=(input_tensor,))
print(f"Model flops: {flops}")
print(f"Model parameters: {params}")

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates